In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col
from pyspark.sql.functions import current_date, current_timestamp, datediff, months_between, to_date

In [4]:
sparkSession = SparkSession.builder.appName('project1').getOrCreate()

### 第六章

In [8]:
df = sparkSession.read.format("csv")\
.option("header", "true").option("inferSchema", "true")\
.load("./data/retail-data/by-day/2010-12-01.csv")

#### 6.5 字符匹配与判断

In [9]:
from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
    return locate(color_string.upper(), column)\
    .cast("boolean")\
    .alias("is_" + c)

In [12]:
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*")) # has to a be Column type
df.select(*selectedColumns).where(expr("is_white OR is_red"))\
.select("is_white","is_red","is_black","Description").show(3, False)

+--------+------+--------+----------------------------------+
|is_white|is_red|is_black|Description                       |
+--------+------+--------+----------------------------------+
|true    |false |false   |WHITE HANGING HEART T-LIGHT HOLDER|
|true    |false |false   |WHITE METAL LANTERN               |
|true    |true  |false   |RED WOOLLY HOTTIE WHITE HEART.    |
+--------+------+--------+----------------------------------+
only showing top 3 rows



#### 6.6 日期与时间戳


In [18]:
dateDF = sparkSession.range(10).withColumn("today", current_date())\
.withColumn("now", current_timestamp())

dateDF.createOrReplaceTempView("dateTable")

In [20]:
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [32]:
dateFormat = "yyyy-dd-MM"
cleanDateDF = sparkSession.range(1).select(
to_date(lit("2017-12-11"), dateFormat).alias("date"),
to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

In [34]:
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()
cleanDateDF.filter(col("date2") > "'2017-12-12'").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+

